# TALLER 9


In [13]:
import random
import math
T = int(1e12) 


def get_random_number():
    return random.random()

def get_exponential(lambda_parameter):
    random_number = get_random_number()
    value = - (math.log(1 - random_number) / float(lambda_parameter))
    return value

def get_normal():
    random_number = get_random_number()
    square_root = math.sqrt(-2 * math.log1p(random_number - 1))
    sin = math.sin(2 * math.pi * random_number)
    value = square_root * sin
    if(value < 0):
        value = 0
    return value


def arrival(actual_simulated_time):
    global time,number_of_clients,number_of_arrivals,arrivals,lambda_parameter, max_service_time, main_times, clients, times
    time = actual_simulated_time
    clients.append(number_of_clients)
    times.append(time)
    number_of_clients += 1
    number_of_arrivals += 1
    arrivals.append(time)
    clients.append(number_of_clients)
    times.append(time)
    new_simulated_time = get_exponential(lambda_parameter)
    if(time + new_simulated_time < max_service_time):
        main_times['arrival_times'] = time + new_simulated_time
    if(number_of_clients == 1):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)
        

def server(actual_simulated_time):
    global time, number_of_clients, number_of_departures, departures, main_times, clients, times
    time = actual_simulated_time
    clients.append(number_of_clients)
    times.append(time)
    number_of_clients -= 1
    number_of_departures += 1
    departures.append(time)
    clients.append(number_of_clients)
    times.append(time)
    if(number_of_clients > 0):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)


def sample_mean(p_med, xi, j):
    return p_med + ( (xi - p_med) / (j + 1.0))
    

def sample_variance(p_med, aver_act, p_variance, j):
    p1 = (1 - ( 1.0 / j)) * p_variance
    p2 = ((aver_act - p_med) ** 2 ) * (j + 1)
    return p1 + p2


lambda_parameter = 3
max_service_time = 10
simulations = [100, 1000, 10000]
for sim in simulations:
    previous_average_time = 0
    average_time = 0
    variance = 0
    
    averages = []
    variances = []
    tiempo_fila = []
    for i in range(sim):
        tiempo_fila.clear()
        time = 0 
        number_of_clients = 0
        number_of_arrivals = 0
        number_of_departures = 0
        
        simulation_time_after_last_client = 0
        actual_simulated_time = 0
        
        mean_system_time = 0
        mean_queue_time = 0
        
        departures = []
        arrivals = []
        services_time = []
        
        clients = [0] # Eje Y
        times = [0]   # Eje X
        
        main_times = {
            'arrival_times': T,
            'departure_times': T
        }
        
        actual_simulated_time = get_exponential(lambda_parameter)
        
        if actual_simulated_time > max_service_time:
            simulation_time_after_last_client = 0
            mean_system_time = 0
            mean_queue_time = 0
        else:
            arrival(actual_simulated_time)
                
            while(main_times['arrival_times'] != T or main_times['departure_times'] != T):
                if(main_times['arrival_times'] < main_times['departure_times']):
                    new_simulated_time = main_times['arrival_times']
                    main_times['arrival_times'] = T
                    arrival(new_simulated_time)
                if(main_times['departure_times'] < main_times['arrival_times']):
                    new_simulated_time = main_times['departure_times']
                    main_times['departure_times'] = T
                    server(new_simulated_time)
                            
            simulation_time_after_last_client = max(0, time - max_service_time)
            accumulated_1 = 0
            accumulated_2 = 0
            counter = 0
            while counter < number_of_arrivals:
                accumulated_1 += departures[counter] - arrivals[counter]
                accumulated_2 += departures[counter] - arrivals[counter] - services_time[counter]
                counter += 1
                            
            mean_system_time = accumulated_1 / number_of_arrivals
            mean_queue_time = accumulated_2 / number_of_arrivals
        if i >= 1:
            average_time = sample_mean(average_time, mean_queue_time, i)
        else:
            average_time = mean_queue_time
            
        if i >= 2:
            variance = sample_variance(
                previous_average_time,
                average_time,
                variance,
                i)
            
        previous_average_time = average_time
        averages.append(average_time)
        variances.append(variance)
        
    print("K={}".format(sim))
    print("Media=",averages[sim-1])
    print("Varianza=",variances[sim-1])
    print("")

K=100
Media= 3.646329894408488
Varianza= 5.322507275217535

K=1000
Media= 3.7728678216260065
Varianza= 4.512533708249727

K=10000
Media= 3.8462600660847115
Varianza= 4.608599005421144

